# Salários dos Juízes do TJDFT - 12/2017

Conforme determinação do Conselho Nacional de Justiça - CNJ ([Portaria n. 63, de 17 de agosto de 2017](http://www.cnj.jus.br/busca-atos-adm?documento=3352)) todos os 90 tribunais brasileiros devem disponibilizar as informações de remuneração dos seus magistrados, em cumprimento à lei de Acesso à Informação. Para padronizar a divulgação desses dados o CNJ criou uma planilha Excel padrão, que deve ser preenchida pelos tribunais, e passou a disponibilizar todas as planilhas em seu site ([Remuneraçao dos Magistrados](http://www.cnj.jus.br/transparencia/remuneracao-dos-magistrados)).

Como todas as planilhas são, supostamente, iguais, iremos criar um conjunto de instruções Python para abrir e importar uma planilha, e analisar os dados contidos nela. Além disso, esse Jupyter Notebook servirá como um tutorial introdutório ao uso da biblioteca **pandas** e à análise de dados com Python. Como exemplo usaremos os dados disponibilizados pelo Tribunal de Justiça do Distrito Federal e Territórios - TJDFT, com base na planilha baixada do site do CNJ no dia 07/12/2017.

Usando esse notebook como exemplo e a planilha de outro tibunal, qualquer pessoa deve conseguir realizar as mesmas análises, ou até mesmo expandir o trabalho realizado aqui. Obviamente algum tribunal específico pode descumprir a determinação do CNJ e acabar por disponibilizar seus dados em um formato diferente da planilha padrão. Se isso acontecer, mudanças deverão ser realizadas no código, principalmente na parte intitulada **Importando os dados**.


## Importando os dados

Inicialmente é preciso acessar a página específica do CNJ ([Remuneraçao dos Magistrados](http://www.cnj.jus.br/transparencia/remuneracao-dos-magistrados)) e baixar o arquivo referente ao Tribunal de Justiça do Distrito Federal e Territórios. Coloque o arquivo .xls na mesma pasta do Jupyter Notebook (arquivo .ipynb).

É importante abrir o arquivo .xls no próprio Excel (ou programa similar) para visualizar o arquivo e entender os dados. Ao abrir o arquivo é possível perceber que existem 5 diferentes planilhas, mas os dados principais estão na primeira planilha, chamada **Contracheque**. Além disso percebemos que as primeiras linhas dessa planilha não contêm dados, mas apenas explicações de preenchimento. O cabeçalho das colunas está nas linhas 19 e 20 (mescladas) e os dados começam da linha 21. A forma mais fácil de resolver esse problema é importar as colunas desejadas, sem cabeçalho, e inserir o cabeçalho posteriormente.

Para cada linha temos 17 colunas, a saber: CPF (que não apresenta dados válidos), **Nome**, **Cargo**, **Lotação**, **Subsídio (R\$)**, Direitos Pessoais(1), Indenizações(2), Direitos Eventuais(3), **Total de Rendimentos(4)**, Previdência Pública(5) (R\$), Imposto de Renda(6) (R\$), Descontos Diversos(7) (R\$), Retenção por Teto Constitucional(8) (R\$), Total de Descontos(9), **Rendimento Líquido(10)**, Remuneração do órgão de origem(11) (R\$) e Diárias(12) (R\$). As colunas em negrito correspondem àquelas que iremos tratar em nossa análise.

Os dados serão lidos do arquivo usando as funções das bibliotecas **pandas** e **xlrd**, que permitem importar facilmente uma ou mais planilhas do arquivo Excel, e também permitem a exclusão de linhas indesejadas. Depois de importados, os dados ficarão disponíveis em um **dataframe**.

In [ ]:
# Importando a biblioteca pandas
import pandas as pd

# Importando o arquivo xls
# somente a planilha Contracheque -> sheet_name="Contracheque"
# descartando o cabecalho -> header=None
# descartando as 20 primeiras linhas -> skiprows=20
# e importando apenas as colunas selecionados -> usecols=[1, 2, 3, 4, 8, 14]
df = pd.read_excel("Salarios_Juizes_TJDFT_122017.xls", sheet_name="Contracheque", header=None,  skiprows=20, usecols=[1, 2, 3, 4, 8, 14])

# Incluindo o cabecalho para as 5 colunas importadas
df.columns = ["Nome", "Cargo", "Lotacao", "Subsidio", "Rendimento_Bruto", "Rendimento_Liquido"]

# Exibindo os 5 primeiros registros importados
df.head()

## Analisando os dados

O pacote **pandas** possui um conjunto de métodos que permitem realizar algumas análises em cima dos dados presentes no **dataframe**. Os comentários em cada célula explicam o que faz cada um dos métodos.

In [ ]:
# O pandas consegue mostrar um resumo quantitativo dos dados contidos no dataframe
# Isso vale apenas para as colunas que contenham valores numéricos
df.describe()

In [ ]:
# Quantos magistrados estão na folha de pagamento do TJDFT?
df.count()

In [ ]:
# É possível contar apenas uma das colunas, fazendo referência direta a
# essa coluna no dataframe e depois executando o método count()
df["Nome"].count() # ou df.Nome.count()

# E também é possível formatar a saída para um texto mais legível
print(f"O TJDFT possui {df.Nome.count()} magistrados")

In [ ]:
# Qual o somatório do salário líquido pago a todos esses magistrados?
df.Rendimento_Liquido.sum()

In [ ]:
# Qual a média salarial dos magistrados do TJDFT?
# O valor foi arrendodado para 2 casas decimais
print(f"Em média, um magistrado do TJDFT recebe R${df.Rendimento_Liquido.mean().round(2)} de salário líquido por mês.")

In [ ]:
# Quais os 5 maiores salários líquidos?
df.nlargest(5, "Rendimento_Liquido")

In [ ]:
# E os 5 menores salários líquidos?
df.nsmallest(5, "Rendimento_Liquido")

In [ ]:
# Vamos separar os magistrados aposentados constantes na folha de pagamento do TJDFT
# Nesse caso específico, quando o magistrado está aposentado esse termo aparece no campo Lotacao
# Isso pode não ser verdade para os dados de outros tribunais
# Vamos colocar o resultado da nossa seleção em outro dataframe
df_aposentados = df[df.Lotacao.str.contains("APOSENTADOS")]

df_aposentados.head()

In [ ]:
# Qual o número de aposentados na folha de pagamento do TJDFT?
df_aposentados.Nome.count()

In [ ]:
# Qual o salário médio dos aposentados do TJDFT?
print(f"No TJDFT um magistrado aposentado recebe, em média, R${df_aposentados.Rendimento_Liquido.mean().round(2)}")

In [ ]:
# Quais os maiores salários de magistrados aposentados?
df_aposentados.nlargest(5, "Rendimento_Liquido")

In [ ]:
# Agora vamos separar os desembargadores constantes na folha de pagamento do TJDFT
# Nesse caso específico, quando o magistrado é desembargador esse termo aparece no campo Cargo
# Isso pode não ser verdade para os dados de outros tribunais
# Vamos colocar o resultado da nossa seleção em outro dataframe
# mas agora devemos ter cuidado para separar apenas os desembargadores que estão na ativa
df_desembargadores = df[(df.Cargo.str.contains("DESEMBARGADOR")) & ~(df.Lotacao.str.contains("APOSENTADOS"))]

df_desembargadores.head()

In [ ]:
# Qual o número de desembargadores na folha de pagamento do TJDFT?
df_desembargadores.Nome.count()

In [ ]:
# Qual o salário médio dos desembargadores do TJDFT?
print(f"No TJDFT um desembargador recebe, em média, R${df_desembargadores.Rendimento_Liquido.mean().round(2)}")

In [ ]:
# Quais os maiores salários de desembargadores?
df_desembargadores.nlargest(5, "Rendimento_Liquido")

## Plotando gráficos

A biblioteca **pandas** tem integração nativa com a biblioteca **matplotlib**, o que permite a plotagem de Series/DataFrames de forma facilitada. Neste caso, apenas chamamos o método hist() na coluna que queremos produzir o histograma e o gráfico é gerado. É importante customizar o gráfico por meio da biblioteca **matplotlib**, sempre lembrando de rotular os eixos.

In [ ]:
# Importando a biblioteca
import matplotlib.pyplot as plt

# e determinando que os graficos serão desenhados no próprio Jupyter Notebook
%matplotlib inline

# Vamos ver a distribuição dos rendimento líquidos
df.Rendimento_Liquido.hist(bins=50)
plt.title("Distribuicao por Salarios")
plt.ylabel('Qtde de Magistrados')
plt.xlabel('Rendimento Líquido');